In [1]:
import pandas as pd
from glob import glob
import numpy as np
from scipy.spatial import distance
import re
import pymesh,math

In [2]:
# !pip install xlrd

In [3]:
data = pd.read_excel("../planilla_colecta.xlsx")

In [4]:
data = data[["id","sexo","peso","promedio altura","promedio cir cad"]]

In [5]:
row_data = data.loc[data['id'] == 1].iloc[0].tolist()

In [6]:
row_data

[1, u'f', 62.299999999999997, 163.40000000000001, 101.045]

In [7]:
fs_landmarks = sorted(glob("../data_artificial_cites/landmarks_reales/*.pts"))
fs_mesh = sorted(glob("./out/torsos/*.ply"))

In [20]:
estatura_error = []
estatura_calculada = []
cadera_calculada=[]
cadera_error=[]
for landmarks_f, mesh_f in zip(fs_landmarks, fs_mesh):
    row_data = data.loc[data['id'] == int(re.findall("\d+",landmarks_f)[0])].iloc[0].tolist()
    landmarks = pd.read_csv(landmarks_f,header=None,index_col=[0],skiprows=range(2),delimiter=r"\s+")
    output_mesh = pymesh.load_mesh(mesh_f)
    #mesh = pymesh.load_mesh(mesh_f)
    head = landmarks.iloc[14].tolist()
    center_foot = np.array([landmarks.iloc[7].tolist(),landmarks.iloc[6].tolist()]).mean(0)
    estatura = distance.pdist(np.array([head,center_foot]))[0]*100
    estatura_calculada.append(estatura)
    estatura_error.append( estatura - row_data[3])
    
    ls0 = landmarks.iloc[0].tolist()
    ls1 = landmarks.iloc[4].tolist()
    ls2 = landmarks.iloc[9].tolist()
    ls3 = landmarks.iloc[10].tolist()
    knee_left = landmarks.iloc[8].tolist()
    knee_right = landmarks.iloc[5].tolist()
    landmarks_torzo = np.array([ls0,ls1,ls2,ls3])
    center_knee = np.array([knee_left,knee_right]).mean(0)

    def distance3D(a, b):
        return math.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2+(a[2]-b[2])**2)

    def distance2D(a, b):
        return math.sqrt( ((b[0]-a[0])**2)  + ((b[1]-a[1])**2) )

    def angle(a,b):
        uv = a[0]*b[0] + a[1]*b[1]+ a[2]*b[2]
        norma_uv = math.sqrt((a[0]**2+a[1]**2+a[2]**2)) * math.sqrt((b[0]**2+b[1]**2+b[2]**2 ))
        return math.acos(uv/norma_uv)

    semi_landmark = landmarks_torzo[1:3].mean(0)
    distancias = []
    for v in output_mesh.vertices:
        distancias.append(distance3D(semi_landmark,v))
    distancias = np.array(distancias)
    ver_semi_landmark = output_mesh.vertices[distancias.argmin()]
    distancias.argmin()

    xdot = (head[0]*center_knee[0] + head[1]*center_knee[1] + head[2]*center_knee[2])
    norma = math.sqrt(head[0]**2+head[1]**2+head[2]**2) * math.sqrt(center_knee[0]**2+center_knee[1]**2+center_knee[2]**2)
    angle_head = (math.cos(xdot/norma) ** -1)
    angle_head_r = math.radians(90 - math.degrees(angle_head))
    angle_head_r

    new_mesh = pymesh.meshutils.generate_icosphere(0.01,ver_semi_landmark)
    q = pymesh.misc.Quaternion(quat=[1,-angle_head_r, 0, 0])
    circle_points = []
    for i in range(360):
        distance_points = 0.15
        angleY = math.radians(i)
        angleZ = 0
        x= ver_semi_landmark[0] + distance_points * math.cos (angleZ) * math.sin(angleY)
        y= ver_semi_landmark[1] + distance_points * math.sin (angleZ)
        z= ver_semi_landmark[2] + distance_points * math.cos (angleZ) * math.cos(angleY)
        v_v = [x,y,z] 
        v = q.rotate(v_v)
        v[2] = v[2] /6.5
        v[1] = v[1] /6.5
        v[0] = v[0] /5.5
        circle_points.append(v)

        #temp = pymesh.meshutils.generate_icosphere(0.01,v)
        #new_mesh = pymesh.boolean(new_mesh, temp , operation="union",engine="auto")
    #pymesh.save_mesh("out/circle.ply", new_mesh)

    circle_points = np.array(circle_points)
    vertexs = []
    for cp in circle_points:
        distancias_circle = []
        for v in output_mesh.vertices:
            distancias_circle.append(distance3D(cp,v))
        vertexs.append(output_mesh.vertices[np.array(distancias_circle).argmin()])
    
    vertexs_2d = np.delete(vertexs, [1], axis=1)
    aux = 0
    for i,v in enumerate(vertexs_2d[:-1]):
        aux = aux + distance2D(v,vertexs_2d[i+1])
    aux = aux + distance2D(vertexs_2d[0],vertexs_2d[-1])
    cadera_calculada.append(aux*100)
    cadera_error.append((aux*100) - row_data[4])

In [9]:
np.shape(estatura_error)

(148,)

In [10]:
data["estatura cal"] = estatura_calculada
data["estatura error"] = estatura_error

In [11]:
np.array(cadera_calculada*100)

array([  97.79377459,  113.5606211 ,   99.07469803, ...,   90.07255579,
         95.22239841,   88.70241601])

In [12]:
data.loc[abs(data['id'])<7]

,id,sexo,peso,promedio altura,promedio cir cad,estatura cal,estatura error
0,1,f,62.3,163.4,101.045,160.664674,-2.735326
1,2,f,81.3,169.6,112.150,167.738174,-1.861826
3,4,f,64.7,172.3,98.400,168.866975,-3.433025
4,5,f,64.1,166.1,94.950,161.785386,-4.314614
5,6,m,79.2,170.6,107.300,165.830201,-4.769799


In [13]:
cadera_calculada

[97.79377459351699,
 113.56062110430078,
 99.07469803473768,
 93.6214220207284,
 95.52199662123043,
 99.01534097097003,
 102.97798799373432,
 90.14125524524601,
 100.77533445379223,
 99.3232641585929,
 97.39247663584788,
 92.82785869634007,
 118.44998269877746,
 101.8556021285766,
 108.10325501717767,
 97.34926613228406,
 96.30615133870369,
 92.33377728231751,
 102.19528454823936,
 98.50809676414863,
 101.81911806864893,
 95.04949227701172,
 100.81119298593542,
 98.94851723049844,
 92.29566094375402,
 103.07264919053239,
 99.98205395719879,
 108.40327546156412,
 97.2914015572941,
 89.54703213008533,
 87.38333976074135,
 94.92386075530975,
 97.05695590282828,
 91.60814441856205,
 103.2631240084873,
 115.49345462947433,
 89.8269311739895,
 92.59891222229238,
 100.43319847313059,
 95.27429661707868,
 103.59627428134128,
 96.39822181600297,
 93.74661400651787,
 98.14129744297014,
 111.08977412286293,
 111.46484627008037,
 106.20913699379382,
 91.9009568157967,
 105.70354726005668,
 94.8684

In [19]:
cadera_error

[-30.106545370525666,
 -12.273825797692666,
 -11.778003378769569,
 -11.039484559083135,
 -10.645401699895913,
 -10.204715451760649,
 -9.7373990316699377,
 -9.5529678699146672,
 -9.2221413036599245,
 -8.7357297251021606,
 -8.7335330903400745,
 -8.4010988049039099,
 -8.3867479304313122,
 -8.3460341122548982,
 -7.8086364723467909,
 -7.5317043119171956,
 -7.3103983554233736,
 -6.7791451755580567,
 -6.5228898330089748,
 -6.1419032358513732,
 -6.1287276240984596,
 -6.0853994087336929,
 -5.99013768605856,
 -5.8161396605044615,
 -5.7975839903982376,
 -5.7479929140649517,
 -5.7037257186587169,
 -5.5883361926589856,
 -5.5179460428012135,
 -5.5166602392586555,
 -5.4442772297888808,
 -5.2364767374234589,
 -4.8159002564476481,
 -4.7408630062061832,
 -4.6634965651732898,
 -4.5895260097892248,
 -4.4500173012225446,
 -4.4077603994445838,
 -4.0730688260105126,
 -4.066638413100236,
 -4.011918560051086,
 -3.8495487329888363,
 -3.8097251498585365,
 -3.7587447547539909,
 -3.6818135205901541,
 -3.4299973106

In [15]:
data["cadera calculada"] = cadera_calculada
data["cadera error"] = cadera_error

In [17]:
data.to_csv("primeros_cacl_1.csv")